## Purpose

Information about house & condo sales between the years 2016-2021 can be found [here](https://assessment.winnipeg.ca/AsmtTax/English/SelfService/SalesBooks.stm) from the City of Winnipeg website, who release new data every 2 years. 

[Tabula](https://tabula.technology/) is then used to extract the content from the given PDF files into their respective CSV files (22 files in total) so that Pandas library can work with it.

The purpose of this notebook is to combine those multiple CSV files into 1 big CSV file, along with that property's tax assessment information. Some simple data clean up is performed as well. 

This data will later be uploaded to a Azure Postgres Database.

### Input

This notebook requires all the CSV files outputted by Tabula so that they can be combined into 1 CSV file, as well as the Winnipeg property tax assessment dataset found [here](https://data.winnipeg.ca/Assessment-Taxation-Corporate/Assessment-Parcels/d4mq-wa44)

### Output

This notebook will output 2 CSV files, with house & condo sale + tax assessment data in separate files , in the ```output``` folder

In [3]:
import pandas as pd
import os

In [4]:
# Store the local paths of each house sale CSV files in a list

house_filepaths_2016_2018 = ['../datasets/input/house_condo/csv-version/2016-2018/' + file \
                for file in os.listdir("/Users/arshb/Desktop/Norima/LeadAI/project/model/datasets/input/house_condo/csv-version/2016-2018") \
                if file.endswith('.csv') and file.startswith('tabula-sales_book_market_region')]

house_filepaths_2019_2021 = ['../datasets/input/house_condo/csv-version/2019-2021/' + file \
                for file in os.listdir("/Users/arshb/Desktop/Norima/LeadAI/project/model/datasets/input/house_condo/csv-version/2019-2021") \
                if file.endswith('.csv') and file.startswith('tabula-sales_book_market_region')]


# Using the above list, we go through each filepath and combine its content into 1 dataframe

combined_house_sale = pd.concat(map(pd.read_csv, house_filepaths_2016_2018 + house_filepaths_2019_2021))

In [5]:
# Same thing as previous cell but combining condo sale data instead

condo_filepath_2016_2018 = '../datasets/input/house_condo/csv-version/2016-2018/tabula-sales_book_condominium.csv'

condo_filepath_2019_2021 = '../datasets/input/house_condo/csv-version/2019-2021/tabula-sales_book_condominium.csv'

combined_condo_sale = pd.concat(map(pd.read_csv, [condo_filepath_2016_2018, condo_filepath_2019_2021]))

In [6]:
# This data will be used to add the property tax assessment information to their respective entries in the house & condo sale data

tax_assessment = pd.read_csv("../datasets/input/tax-assessment-details.csv")

C:\Users\arshb\AppData\Local\Temp\ipykernel_7588\2642005905.py:3: DtypeWarning: Columns (2,3,4,39,40,42,43,45,46,56,57,59,60,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  tax_assessment = pd.read_csv("../datasets/input/tax-assessment-details.csv")


In [7]:
# Inspect the content of house sale dataframe

combined_house_sale.sample(3)

,Property Address,Roll Number,Building Type,Sale Year,Sale Month,Sale Price,Time Adjust Sale Price
453,1096 CHURCH AVE,14051767000,OS-One Storey,2021,3,"$247,000","$247,000"
641,124 HILL ST,6081202000,OS-One Storey,2019,9,"$260,000","$282,600"
721,27 ALBURG DR,8005911900,OS-ONE STOREY,2016,9,"$280,000","$289,500"


In [8]:
# Combined condo sale dataframe has 1 extra column (Condominium number) and 1 fewer column (Building Type) than house sale dataframe:

combined_condo_sale.sample(3)

,Condominium Number,Property Address,Roll Number,Sale Year,Sale Month,Sale Price,Time Adjust Sale Price
646,367.0,10 - 1442 DAKOTA ST,8007868400,2020,10,"$176,000","$176,000"
1716,175.0,506 - 640 MATHIAS AVE,11050756600,2019,11,"$150,000","$150,000"
125,180.0,1210 - 1044 BAIRDMORE BLVD,3060128000,2017,10,"$136,250","$136,900"


In [9]:
# Check which columns the tax assessment data share with house & condo sale data

# They share the following columns:
# 1. Roll Number
# 2. Property Address
# 3. Building type

print('All columns of tax assessment data:')
print(tax_assessment.columns.values)


All columns of tax assessment data:
['Roll Number' 'Street Number' 'Unit Number' 'Street Suffix'
 'Street Direction' 'Street Name' 'Street Type' 'Full Address'
 'Neighbourhood Area' 'Market Region' 'Total Living Area' 'Building Type'
 'Basement' 'Basement Finish' 'Year Built' 'Rooms' 'Air Conditioning'
 'Fire Place' 'Attached Garage' 'Detached Garage' 'Pool'
 'Number Floors Condo' 'Property Use Code' 'Assessed Land Area'
 'Water Frontage Measurement' 'Sewer Frontage Measurement'
 'Property Influences' 'Zoning' 'Total Assessed Value'
 'Total Proposed Assessment Value' 'Assessment Date' 'Detail URL'
 'Current Assessment Year' 'Property Class 1' 'Status 1'
 'Assessed Value 1' 'Property Class 2' 'Status 2' 'Assessed Value 2'
 'Property Class 3' 'Status 3' 'Assessed Value 3' 'Property Class 4'
 'Status 4' 'Assessed Value 4' 'Property Class 5' 'Status 5'
 'Assessed Value 5' 'Proposed Assessment Year' 'Proposed Assessment Date'
 'Proposed Property Class 1' 'Proposed Status 1'
 'Proposed Asses

In [10]:
# Check the data types each column holds of house & condo sale dataframe

data_types = combined_house_sale.dtypes

print('Data types of each column of house sale data:')
print(data_types)

data_types = combined_condo_sale.dtypes

print('\nData types of each column of condo sale data:')
print(data_types)

# All columns of house sale data are object data type, which means they are stored as strings
# Since we will be converting the row content into a JSON object later on, we don't need to worry about datatypes just yet

Data types of each column of house sale data:
Property Address          object
Roll Number               object
Building Type             object
Sale Year                 object
Sale Month                object
Sale Price                object
Time Adjust Sale Price    object
dtype: object

Data types of each column of condo sale data:
Condominium Number        float64
Property Address           object
Roll Number                 int64
Sale Year                   int64
Sale Month                  int64
Sale Price                 object
Time Adjust Sale Price     object
dtype: object


In [11]:
num_missing_vals = combined_house_sale.isnull().sum()

print('Number of missing (NaN) values in each column of house sale data:')
print(num_missing_vals)

num_missing_vals = combined_condo_sale.isnull().sum()

print('\nNumber of missing (NaN) values in each column of condo sale data:')
print(num_missing_vals)


Number of missing (NaN) values in each column of house sale data:
Property Address          0
Roll Number               0
Building Type             0
Sale Year                 0
Sale Month                0
Sale Price                0
Time Adjust Sale Price    0
dtype: int64

Number of missing (NaN) values in each column of condo sale data:
Condominium Number        1
Property Address          0
Roll Number               0
Sale Year                 0
Sale Month                0
Sale Price                0
Time Adjust Sale Price    0
dtype: int64


In [12]:
# Note the unique values of Building Type, Sale Year and Sale Month columns to make sure no valuable content is deleted while cleaning up data in the next cell

print('House Sale data unique values:')
print(pd.concat([combined_house_sale['Building Type'], combined_house_sale['Sale Year'], combined_house_sale['Sale Month']]).unique())

print('\nCondo Sale data unique values:')
print(pd.concat([combined_condo_sale['Sale Year'], combined_condo_sale['Sale Month']]).unique())

House Sale data unique values:
['OS-ONE STOREY' 'OH-ONE & 1/2 STOREY' 'O3-ONE & 3/4 STOREY'
 'TS-TWO STOREY' 'BL-BI-LEVEL' 'FL-4 LEVEL SPLIT' 'TH-TWO & 1/2 STOREY'
 'TL-3 LEVEL SPLIT' 'TO-TWO/ONE STOREY' 'Building Type' 'BL-Bi-Level'
 'OS-One Storey' 'OH-One & 1/2 Storey' 'TS-Two Storey'
 'O3-One & 3/4 Storey' 'TL-3 Level Split' 'FL-4 Level Split'
 'TH-Two & 1/2 Storey' 'CO-Cabover' 'TO-Two/One Storey' 'T3-Three Storey'
 2017 2016 2018 '2018' '2016' '2017' 'Sale Year' 2020 2019 2021 9 7 10 5 6
 3 4 8 12 2 1 11 '1' '10' '11' '7' '4' '6' '12' '8' '3' '2' '9' '5'
 'Sale Month']

Condo Sale data unique values:
[2017 2016 2018 2020 2019 2021    8   12    7    6    9    1    4    5
    3   11   10    2]


In [13]:
# Note: sometimes Tabula adds the column names as rows in the CSV file, so we remove those rows where they contain 'useless' content

# Remove the rows where BuildingType column has its name as an entry (indicating that the row was added by Tabula on error)
combined_house_sale = combined_house_sale.drop(combined_house_sale[combined_house_sale['Building Type'] == 'Building Type'].index)

# Remove the single pesky row which has NaN value in the condo sale data
combined_condo_sale = combined_condo_sale.drop(combined_condo_sale[combined_condo_sale['Condominium Number'].isnull()].index)


In [14]:
# Check the unique values again to ensure that only the useless or Nan information was removed and nothing else

print('House sale data unique values: ')
print(pd.concat([combined_house_sale['Building Type'], combined_house_sale['Sale Year'], combined_house_sale['Sale Month']]).unique())

print('\nCondo Sale data unique values:')
print(pd.concat([combined_condo_sale['Sale Year'], combined_condo_sale['Sale Month']]).unique())


# Check no NaN values exist after data clean up

num_missing_vals = combined_condo_sale.isnull().sum()

print('\nNumber of missing values in each column of condo sale data:')
print(num_missing_vals)


House sale data unique values: 
['OS-ONE STOREY' 'OH-ONE & 1/2 STOREY' 'O3-ONE & 3/4 STOREY'
 'TS-TWO STOREY' 'BL-BI-LEVEL' 'FL-4 LEVEL SPLIT' 'TH-TWO & 1/2 STOREY'
 'TL-3 LEVEL SPLIT' 'TO-TWO/ONE STOREY' 'BL-Bi-Level' 'OS-One Storey'
 'OH-One & 1/2 Storey' 'TS-Two Storey' 'O3-One & 3/4 Storey'
 'TL-3 Level Split' 'FL-4 Level Split' 'TH-Two & 1/2 Storey' 'CO-Cabover'
 'TO-Two/One Storey' 'T3-Three Storey' 2017 2016 2018 '2018' '2016' '2017'
 2020 2019 2021 9 7 10 5 6 3 4 8 12 2 1 11 '1' '10' '11' '7' '4' '6' '12'
 '8' '3' '2' '9' '5']

Condo Sale data unique values:
[2017 2016 2018 2020 2019 2021    8   12    7    6    9    1    4    5
    3   11   10    2]

Number of missing values in each column of condo sale data:
Condominium Number        0
Property Address          0
Roll Number               0
Sale Year                 0
Sale Month                0
Sale Price                0
Time Adjust Sale Price    0
dtype: int64


In [15]:
# Check if duplicate entries exist (since the property can be sold multiple times between 2016-2021)

duplicates = combined_house_sale['Roll Number'].duplicated(keep=False)

# Sort the duplicate entries by their property address so we can view them easily besides each other

combined_house_sale.loc[duplicates, :].sort_values('Property Address')

# Note: it was decided we keep the duplicate entries as they are still separate sale transactions that occurred independently, 
# and actually help our machine learning model by providing it more training data

,Property Address,Roll Number,Building Type,Sale Year,Sale Month,Sale Price,Time Adjust Sale Price
1307,1 ALDRICH AVE,7562070000,OS-One Storey,2019,12,"$260,000","$287,000"
1308,1 ALDRICH AVE,7562070000,OS-One Storey,2020,6,"$358,500","$385,400"
483,1 ECHO BAY,6022329000,OS-One Storey,2020,12,"$340,000","$347,500"
516,1 ECHO BAY,6022329000,OS-ONE STOREY,2016,6,"$295,000","$306,800"
619,1 NEWARK RD,6043504000,OS-ONE STOREY,2018,1,"$366,000","$367,500"
...,...,...,...,...,...,...,...
509,99 PAULLEY DR,9003063500,OS-ONE STOREY,2017,5,"$293,000","$298,600"
616,99 WOODFIELD BAY,1000674900,OS-One Storey,2020,5,"$349,000","$371,000"
643,99 WOODFIELD BAY,1000674900,OS-ONE STOREY,2017,12,"$320,000","$321,600"
1091,992 MONCTON AVE,2051970000,OS-ONE STOREY,2016,6,"$203,100","$209,600"


In [16]:
# Combine the property tax assessment information for each row in the house & condo sale data

# Columns from house & condo sale data to be combined with its respective row in the tax assessment data
columns_to_use = ['Roll Number', 'Sale Year', 'Sale Month', 'Sale Price', 'Time Adjust Sale Price']

# Perform left join where the roll numbers match between house & condo sale data and tax assessment data
house_sale_with_tax = combined_house_sale[columns_to_use].merge(tax_assessment, how='left', on='Roll Number')
condo_sale_with_tax = combined_condo_sale[columns_to_use + ['Condominium Number']].merge(tax_assessment, how='left', on='Roll Number')

# Check the left join was successful by comparing the dataframe length

print('Size of house sale data: ' + str(len(combined_house_sale)))
print('Size of combined house sale and tax details dataframe: ' + str(len(house_sale_with_tax)))

print('Size of condo sale data: ' + str(len(combined_condo_sale)))
print('Size of combined condo sale and tax details dataframe: ' + str(len(condo_sale_with_tax)))

Size of house sale data: 38999
Size of combined house sale and tax details dataframe: 38999
Size of condo sale data: 8386
Size of combined condo sale and tax details dataframe: 8386


In [17]:
# Inspect the final dataframe

house_sale_with_tax.sample(3)

,Roll Number,Sale Year,Sale Month,Sale Price,Time Adjust Sale Price,Street Number,Unit Number,Street Suffix,Street Direction,Street Name,...,Proposed Status 3,Proposed Assessment Value 3,Proposed Property Class 4,Proposed Status 4,Proposed Assessment Value 4,Proposed Property Class 5,Proposed Status 5,Proposed Assessment Value 5,Multiple Residences,Geometry
22196,8007855600,2021,2,"$316,500","$318,700",554.0,NaN,NaN,NaN,NOVAVISTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,MULTIPOLYGON (((-97.090974958125 49.8246429915...
9866,13090472100,2016,6,"$160,000","$166,900",1616.0,NaN,NaN,NaN,LOGAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,MULTIPOLYGON (((-97.191248083283 49.9199237000...
16239,3043189000,2016,5,"$269,000","$287,800",39.0,NaN,NaN,NaN,MAGDALENE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,MULTIPOLYGON (((-97.152118138833 49.7897224407...


In [18]:
# Output both house sale & condo sale dataframe to separate CSV

house_sale_with_tax.to_csv('../datasets/output/house_sale_with_tax.csv', index=False)
condo_sale_with_tax.to_csv('../datasets/output/condo_sale_with_tax.csv', index=False)
